<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>ABOUT</b></p>
</div>

This is yet another LGB notebook in the JPX competition

Its main difference is the modular structure:

- An utility script [jpx_feat_util](https://www.kaggle.com/code/notabene/jpx-feat-util) deals with the pre-processing & feature engineering
- This script is used by a notebook [jpx_feat_generation](https://www.kaggle.com/code/notabene/jpx-feat-generation) which deals with the CV. It pre-computes the CV fold files which can then be used by simply importing the output of this notebook
- Another notebook [jpx-lgb-train](https://www.kaggle.com/code/notabene/jpx-lgb-train) imports the fold files and trains a separate LGB model on each fold
- The current notebook then imports the models and the utility script to produce a submission file

In [ ]:
import numpy as np
import pandas as pd
import lightgbm
import pickle
import jpx_tokyo_market_prediction
from jpx_feat_util import get_stock_features, create_features, drop_cols, fill_nan_inf

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

supp = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')
supp,models = supp[supp.Open.notnull() & supp.Target.notnull()],[]
for i in range(5):
    models.append(pickle.load(open(f'../input/jpx-lgb-train/lgb_model{i}', "rb")))

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    test_date = max(prices.Date) #should be just a single Date
    prices = pd.concat([supp, fill_nan_inf(prices)])
    features = create_features(prices)
    X_test = features[features.Date == test_date]
    print('len X_test = ' + str(len(X_test)))
    X_test = X_test.drop(drop_cols, axis=1)
    lgbm_preds = []
    for model in models:
        lgbm_preds.append( model.predict(X_test) )
    lgbm_preds = np.mean(lgbm_preds, axis=0)
    sample_prediction["Prediction"] = lgbm_preds
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction['Rank'] = np.arange(0,len(prices))
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>CREDITS</b></p>
</div>

This modular structure used here is influenced by [this notebook](https://www.kaggle.com/code/slawekbiel/short-fast-nn-lgb) of [Slawek Biel](https://www.kaggle.com/slawekbiel).

The layout used in the markup cells of this notebook is influenced by [this notebook](https://www.kaggle.com/code/shtrausslearning/building-an-asset-trading-strategy) of [Andrey Shtrauss](https://www.kaggle.com/shtrausslearning).